In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

In [14]:
class LeNet(nn.Module):
    def __init__(self, num_classes=14):
        super(LeNet, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        
        # Placeholder for dynamically calculated input features for fc1
        self.fc1_input_features = None
        
        # Fully connected layers (fc1's input size will be adjusted after shape calculation)
        self.fc1 = nn.Linear(1, 120)  # Temporary placeholder, will update after shape calculation
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        # Apply convolution layers with pooling and activation
        x = nn.ReLU()(self.conv1(x))
        x = nn.MaxPool2d(kernel_size=2, stride=2)(x)
        
        x = nn.ReLU()(self.conv2(x))
        x = nn.MaxPool2d(kernel_size=2, stride=2)(x)
        
        # Determine the flattened feature size if not already done
        if self.fc1_input_features is None:
            self.fc1_input_features = x.view(x.size(0), -1).size(1)
            self.fc1 = nn.Linear(self.fc1_input_features, 120)  # Adjust fc1 with correct input features
            
        # Flatten and apply fully connected layers with ReLU
        x = torch.flatten(x, 1)
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        x = self.fc3(x)
        
        return x


In [15]:
batch_size = 64
learning_rate = 0.001
num_epochs = 20

# Define your custom dataset directory paths
train_dir = '../data/Train'
test_dir = '../data/Test'

In [16]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize images to 32x32 for LeNet
    transforms.Grayscale(num_output_channels=1),  # Ensure single-channel grayscale
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),  # Normalize for grayscale images
])

# Load your custom dataset
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [17]:
model = LeNet(num_classes=14)  # Adjusted for 14 output classes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [18]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

Epoch [1/20], Loss: 1.5554
Epoch [2/20], Loss: 1.0518
Epoch [3/20], Loss: 0.9451
Epoch [4/20], Loss: 0.8859
Epoch [5/20], Loss: 0.8340
Epoch [6/20], Loss: 0.7955
Epoch [7/20], Loss: 0.7660
Epoch [8/20], Loss: 0.7380
Epoch [9/20], Loss: 0.7158
Epoch [10/20], Loss: 0.6918
Epoch [11/20], Loss: 0.6751
Epoch [12/20], Loss: 0.6609
Epoch [13/20], Loss: 0.6385
Epoch [14/20], Loss: 0.6255
Epoch [15/20], Loss: 0.6129
Epoch [16/20], Loss: 0.5968
Epoch [17/20], Loss: 0.5843
Epoch [18/20], Loss: 0.5723
Epoch [19/20], Loss: 0.5573
Epoch [20/20], Loss: 0.5490


In [19]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')

Test Accuracy: 73.48%


In [20]:
torch.save(model.state_dict(), "lenet_model.pth")
print("Model saved as lenet_model.pth")

Model saved as lenet_model.pth
